<a href="https://colab.research.google.com/github/SusheelThapa/ML-From-Scratch/blob/tfProject/tensorflow/projects/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SMS Text Classification

## Importing the necessary packages

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


## Getting dataset

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-05-10 06:44:26--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.004s  

2023-05-10 06:44:26 (81.3 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-05-10 06:44:26--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

## Loading Dataset into dataframe

In [3]:
col_names = ['label','messages']

# Adding sep="\t" as data is in tsv format
train_dataset = pd.read_csv(train_file_path,sep='\t' ,names=col_names)
test_dataset = pd.read_csv(test_file_path,sep='\t', names=col_names)

## Preprocessing Dataset

When we look into our data using `head()` method we can see that we have two column "messages" and "label". And label define whether our message is spam or ham.

We will encode ham as '0' and spam as '1'.

In [4]:
label_dict = {
    "ham":0,
    "spam":1
}

train_dataset['msg_type'] = train_dataset['label'].map(label_dict)
test_dataset['msg_type'] = train_dataset['label'].map(label_dict)

In [6]:
train_X = train_dataset['messages']
train_y = train_dataset['msg_type']

test_X = test_dataset['messages']
test_y = test_dataset['msg_type']

## Tokenization


In [22]:
vocab_size = 700 # Setting some arbitary value

tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=vocab_size,
    char_level=False,
    oov_token="",
)

tokenizer.fit_on_texts(train_X)

In [23]:
word_index = tokenizer.word_index
print("Unique Words= ",len(word_index))

Unique Words=  7693


## Sequencing and Padding

In [24]:
train_X_seq = tokenizer.texts_to_sequences(train_X)
train_padding = keras.preprocessing.sequence.pad_sequences(train_X_seq, maxlen=50, 
                                                             padding="post", truncating="post")
test_X_seq = tokenizer.texts_to_sequences(test_X)
test_padding = keras.preprocessing.sequence.pad_sequences(test_X_seq, maxlen=50, 
                                                             padding="post", truncating="post")

In [10]:
train_padding.shape

(4179, 50)

## Creating a model

In [41]:

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, 50,input_length=50),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(24, activation='relu'),
  tf.keras.layers.Dense(1,activation='sigmoid')])


In [42]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 50, 50)            35000     
                                                                 
 global_average_pooling1d_1   (None, 50)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_5 (Dense)             (None, 24)                1224      
                                                                 
 dense_6 (Dense)             (None, 1)                 25        
                                                                 
Total params: 36,249
Trainable params: 36,249
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.compile(optimizer='adam',loss="binary_crossentropy", metrics=['accuracy'])

In [44]:
model.fit(train_padding, train_y,epochs=20,validation_data=(test_padding,test_y))

Epoch 1/20
131/131 [==============================] - 11s 73ms/step - loss: 0.5098 - accuracy: 0.8382 - val_loss: 0.3742 - val_accuracy: 0.8807
Epoch 2/20
131/131 [==============================] - 3s 21ms/step - loss: 0.2663 - accuracy: 0.8660 - val_loss: 0.4280 - val_accuracy: 0.8807
Epoch 3/20
131/131 [==============================] - 2s 16ms/step - loss: 0.1643 - accuracy: 0.9390 - val_loss: 0.5810 - val_accuracy: 0.7974
Epoch 4/20
131/131 [==============================] - 1s 11ms/step - loss: 0.0880 - accuracy: 0.9725 - val_loss: 0.7637 - val_accuracy: 0.7931
Epoch 5/20
131/131 [==============================] - 1s 8ms/step - loss: 0.0599 - accuracy: 0.9818 - val_loss: 0.9084 - val_accuracy: 0.7909
Epoch 6/20
131/131 [==============================] - 1s 9ms/step - loss: 0.0475 - accuracy: 0.9854 - val_loss: 1.0278 - val_accuracy: 0.7874
Epoch 7/20
131/131 [==============================] - 1s 9ms/step - loss: 0.0409 - accuracy: 0.9871 - val_loss: 1.1196 - val_accuracy: 0.7874
E

In [45]:
model.evaluate(test_padding, test_y)

44/44 [==============================] - 0s 6ms/step - loss: 1.9775 - accuracy: 0.7845


[1.9774609804153442, 0.7844827771186829]

## Predict Function

In [46]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    pred_text_tokenize = tokenizer.texts_to_sequences([pred_text])
    pred_text_padding =  keras.preprocessing.sequence.pad_sequences(pred_text_tokenize, maxlen=50, 
                                                             padding="post", truncating="post")

    predict = model.predict(pred_text_padding)[0][0]

    if predict>0.5:
        prediction = [predict,'spam']
    else:
        prediction = [predict,'ham']

    return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 48ms/step
[0.00028342954, 'ham']


## Test from freecode camp

In [48]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 19ms/step
You passed the challenge. Great job!
